```
Copyright 2023 ServiceNow
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

This notebook optimize a simple Mixed-Integer Programming model with data taken from a probabilistic forecast of the `solar_10min` dataset, to be compared with the same data after errors have been added to the forecast.
It then compute the ability of some metrics to detect said errors.

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import sklearn
import scipy
from ror.experiments import h0_vs_h1 as exp_def

In [ ]:
forecast_file = "./data/forecast_solar.npy"
forecast = np.load(forecast_file)

In [ ]:
# forecast = forecast.clip(min=0)
# forecast = forecast / forecast.mean(axis=(0,1), keepdims=True)

In [ ]:
def do_nothing(samples):
    return samples

def break_correlations(samples):
    return np.random.default_rng().permuted(samples, axis=0)

def permute_units(samples):
    return np.random.default_rng().permutation(samples, axis=2)

def permute_timesteps(samples):
    return np.random.default_rng().permutation(samples, axis=1)

def multiply_data(factor):
    def func(samples):
        return samples * factor
    return func

def increment_data(increment):
    def func(samples):
        return (samples + increment).clip(min=0)
    return func

In [ ]:
def optimize_linprog(forecast, regret_coef, max_activate, verbose=True):
    NUM_SCEN = forecast.shape[0]
    NUM_TIME = forecast.shape[1]
    NUM_UNIT = forecast.shape[2]
    
    # Indices of variables:
    # * How much of the unit `i` is activate: index = `i`
    # * How much power are we selling at time `t`: index = `NUM_UNIT + t`
    # * How much power are we missing at time `t` in scenario `v`: index = `NUM_UNIT + NUM_TIME + NUM_SCEN * t + v`
    TOTAL_VAR = NUM_UNIT + NUM_TIME + NUM_SCEN * NUM_TIME
    
    # Objective (minimization):
    # Negative sum of sold power for all times `t`;
    # plus sum of missing power for all times `t` and scenarios `v`,
    # multiplied by a regret aversion coefficient and divided by the number of scenarios.
    c = np.zeros(TOTAL_VAR)
    c[NUM_UNIT:NUM_UNIT+NUM_TIME] = -1
    c[NUM_UNIT+NUM_TIME:] = regret_coef / NUM_SCEN
    
    # Bounds:
    # * Unit activation: `[0,1]`
    # * Sold power: `[0,infinity]`
    # * Missing power: `[0,infinity]`
    bounds = [(0,1) for _ in range(NUM_UNIT)] + \
        [(0,None) for _ in range(NUM_TIME)] + \
        [(0,None) for _ in range(NUM_SCEN * NUM_TIME)]
    
    # Maximum activate constraint:
    # Sum of all unit activation must be below some constant.
    A_ub_max_activate = np.zeros((1, TOTAL_VAR))
    b_ub_max_activate = np.zeros(1)
    A_ub_max_activate[0,0:NUM_UNIT] = 1
    b_ub_max_activate[0] = max_activate
    
    # Underproduction constraints:
    # Declared production minus missed production, minus sum of each unit production times its activation,
    # must be non-positive.
    A_ub_underprod = np.zeros((NUM_SCEN * NUM_TIME, TOTAL_VAR))
    b_ub_underprod = np.zeros(NUM_SCEN * NUM_TIME)

    for t in range(NUM_TIME):
        for v in range(NUM_SCEN):
            A_ub_underprod[NUM_SCEN * t + v, NUM_UNIT + t] = 1
            A_ub_underprod[NUM_SCEN * t + v, NUM_UNIT + NUM_TIME + NUM_SCEN * t + v] = -1
            for i in range(NUM_UNIT):
                A_ub_underprod[NUM_SCEN * t + v, i] = -forecast[v, t, i]
            b_ub_underprod[NUM_SCEN * t + v] = 0
            
    A_ub = np.concatenate([A_ub_max_activate, A_ub_underprod], axis=0)
    b_ub = np.concatenate([b_ub_max_activate, b_ub_underprod], axis=0)
    
    result = linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=bounds,
        method="highs"
    )
    
    if verbose:
        print("Objective value:", -result.fun)
    return result.x[0:NUM_UNIT], result.x[NUM_UNIT:NUM_UNIT+NUM_TIME]

In [ ]:
def compute_expected_objective(forecast, regret_coef, activations, sales):
    production = (forecast * activations[None,None,:]).sum(axis=2)
    missing_prod = (sales[None,:] - production).clip(min=0)
    return (sales - regret_coef * missing_prod).sum(axis=1)

In [ ]:
def perfect_foresight_objective(forecast, regret_coef, max_activations):
    result = 0    
    for scenario in range(forecast.shape[0]):
        single_forecast = forecast[scenario:scenario+1,:,:]
        act, sl = optimize_linprog(single_forecast, regret_coef, max_activations, verbose=False)
        result = result + compute_expected_objective(single_forecast, regret_coef, act, sl).item()
    return result / forecast.shape[0]

In [ ]:
def measure_loss_of_profit(forecast_gt, transformation_fcst, regret_coef, max_activations, num_trials=1):
    act, sl = optimize_linprog(forecast_gt, regret_coef, max_activations, verbose=False)
    values_gt = compute_expected_objective(forecast_gt, regret_coef, act, sl)
    
    values_fcst = np.zeros(0)
    for _ in range(num_trials):
        forecast_fcst = transformation_fcst(forecast_gt)
        act, sl = optimize_linprog(forecast_fcst, regret_coef, max_activations, verbose=False)
        temp = compute_expected_objective(forecast_gt, regret_coef, act, sl)
        values_fcst = np.concatenate([values_fcst, temp])
    
    is_fcst = np.concatenate([np.zeros(len(values_gt)), np.ones(len(values_fcst))])
    values = np.concatenate([values_gt, values_fcst])
    
    perfect_foresight = perfect_foresight_objective(forecast_gt, regret_coef, max_activations)
    print("Perfect foresight =", perfect_foresight)
    print("Mean obj (GT)     =", values_gt.mean())
    print("Mean obj (FCST)   =", values_fcst.mean())
    print("Normalized (GT)   =", values_gt.mean() / perfect_foresight)
    print("Normalized (FCST) =", values_fcst.mean() / perfect_foresight)
    print("Loss of profit    =", 1 - values_fcst.mean() / values_gt.mean())

In [ ]:
REGRET_COEF = 10.0
MAX_ACTIVATIONS = 50

In [ ]:
%%time
measure_loss_of_profit(
    forecast,
    break_correlations,
    REGRET_COEF,
    MAX_ACTIVATIONS,
    num_trials=10,
)

In [ ]:
%%time
measure_loss_of_profit(
    forecast,
    multiply_data(1.05),
    REGRET_COEF,
    MAX_ACTIVATIONS,
)

In [ ]:
%%time
measure_loss_of_profit(
    forecast,
    increment_data(0.05),
    REGRET_COEF,
    MAX_ACTIVATIONS,
)

# Compare with metrics

Only metrics which can be computed from a distribution sample are included here. The negative log-likelihood requires the functional form of the model used to generate said distribution. And while we kept a copy of the code and weights of the model, it did not use a very stable code base due to very strict library dependancies to simply run. So we have omitted them from this release.

In [ ]:
SELECTION = ["crps_quantile", "energy_1", "variogram_1"]
METRICS = {k: v for k, v in exp_def.METRIC_FUNCTIONS.items() if k in SELECTION}
TRANSFORMATIONS = {
    "ground truth": do_nothing,
    "break correlations": break_correlations,
    "multiply by 1.05": multiply_data(1.05),
    "add 0.05": increment_data(0.05),
}

In [ ]:
def get_stat_power(gt_score, fcst_score):
    diffs = gt_score - fcst_score
    d_mean = diffs.mean()
    d_std = diffs.std()
    
    alpha = 0.05
    num_draws = 7
    threshold = num_draws**0.5 * d_std * scipy.stats.norm.ppf(alpha)
    logsf = scipy.stats.norm.logsf((threshold - num_draws * d_mean) / (num_draws**0.5 * d_std))
    if (~np.isnan(logsf)):
        base10sf = logsf / np.log(10)
        exponent = int(np.floor(base10sf))
        mantisse = 10 ** (base10sf - exponent)
        cdf = scipy.stats.norm.cdf((threshold - num_draws * d_mean) / (num_draws**0.5 * d_std))
        return f"1 - {mantisse} * 10^{exponent} = {cdf}"
    else:
        return "nan"

In [ ]:
def all_metrics_df(forecast_gt, transformations, metrics, num_trials=1):
    results = []
    for metric_name, metric in metrics.items():
        for transformation_name, transformation in transformations.items():
            for gt_index in range(forecast_gt.shape[0]):
                # Remove the ground-truth from the forecast, since this is a pretty huge bias for some metrics
                forecast_without_gt = np.concatenate([forecast_gt[:gt_index], forecast_gt[gt_index+1:]])
                for trial in range(num_trials):
                    forecast_fcst = transformation(forecast_without_gt)
                    value = metric(forecast_gt[gt_index], forecast_fcst)
                    results.append({
                        "metric": metric_name,
                        "transformation": transformation_name,
                        "trial": trial + 1,
                        "entry": gt_index + 1,
                        "value": value,
                    })
    return pd.DataFrame(results)

In [ ]:
%%time
full_df = all_metrics_df(forecast[:, :, :], TRANSFORMATIONS, METRICS)

In [ ]:
for metric in METRICS:
    for transformation in TRANSFORMATIONS:
        if transformation != "ground truth":
            stat_power = get_stat_power(
                full_df[(full_df.transformation == "ground truth") & (full_df.metric == metric)].set_index("entry").value,
                full_df[(full_df.transformation == transformation) & (full_df.metric == metric)].set_index("entry").value,
            )
            print(f"{metric}, {transformation}: Stat Power = {stat_power}")